In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Dados

In [ ]:
# Importar dados
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSr7tU5tK8cvvR32yypE1PArgXhmNbFJ9bw8w6Sm2zQhyaMs27csoo-77vgFedHw9z25Ez3Qm-geKyU/pub?gid=43084471&single=true&output=csv', skiprows= 1)

df.drop(columns='Unnamed: 0', inplace= True)
df.reset_index(drop = True)
df

In [ ]:
df = df[(df["Parameter"] == "E175") | (df["Parameter"] == "RJ70") | (df["Parameter"] == "Fokker 100") | (df["Parameter"] == "B717-200") | (df["Parameter"] == "ARJ21-700") | (df["Parameter"] == "A220") | (df["Parameter"] == "CRJ900")]

## Matriz de correlação

In [ ]:
# Calculando a matriz de correlação
cols = df.columns
string_columns = df.select_dtypes(include=['object']).columns

correlation_matrix = df.drop(columns=string_columns).corr()

plt.figure(figsize=(100, 100))

# Criando um heatmap da matriz de correlação
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')

# Exibindo o gráfico
#plt.show()

# Filtrando pares de colunas com correlação superior a 0.8
# Criando um DataFrame booleano para detectar correlações superiores a 0.8
high_corr_pairs = []

for col1 in correlation_matrix.columns:
    for col2 in correlation_matrix.columns:
        if col1 != col2 and abs(correlation_matrix[col1][col2]) > 0.8:
            high_corr_pairs.append((col1, col2, correlation_matrix[col1][col2]))

# Exibindo os pares de colunas com correlação > 0.8
print("\nPares de colunas com correlação > 0.8:")
for pair in high_corr_pairs:
    print(f"{pair[0]} e {pair[1]}: {pair[2]:.2f}")



Pares de colunas com correlação > 0.8:
MTOW (kg) e Number_of_passengers: 0.90
MTOW (kg) e Wing_area (m²): 0.89
MTOW (kg) e Wing_area (ft²): 0.89
MTOW (kg) e VT_taper_ratio: 0.95
Number_of_passengers e MTOW (kg): 0.90
Number_of_passengers e HT_aspect_ratio: -0.90
Number_of_passengers e VT_aspect_ratio: -0.84
Number_of_passengers e VT_taper_ratio: 0.87
Baggage_compartment_volume (m³) e Baggage_compartment_volume (ft³): 1.00
Baggage_compartment_volume (ft³) e Baggage_compartment_volume (m³): 1.00
Range (NM) e VT_sweep: 0.87
Range (NM) e CVT: -0.82
Cruise_Mach e Wing_taper_ratio: -0.94
Cruise_Mach e Wing_sweep(deg): 0.93
Cruise_Mach e Wing_dihedral (deg): 0.98
Cruise_Mach e Fuselage_length (m): 0.84
Cruise_Mach e Main_section_length(m): 0.89
Cruise_Mach e Number_of_engines: -0.93
Cruise_Mach e Engine_TSFC: 0.94
Takeoff_distance(m) e Landing_distance(m): 0.90
Takeoff_distance(m) e HT_sweep (deg): 0.89
Takeoff_distance(m) e VT_sweep: 0.94
Takeoff_distance(m) e Number_of_doors: -0.94
Takeoff

### Função para fazer os plots

In [ ]:
def plotar(coluna1, coluna2):

    df[coluna1] = df[coluna1].astype(str).str.replace(',', '').str.strip()
    df[coluna2] = df[coluna2].astype(str).str.replace(',', '').str.strip()

    # Converter para float
    df[coluna1] = pd.to_numeric(df[coluna1], errors='coerce')
    df[coluna2] = pd.to_numeric(df[coluna2], errors='coerce')

    X = df[coluna1].values
    Y = df[coluna2].values
    
    # Ajuste polinomial de grau 1
    Z = np.polyfit(X, Y, 1)
    p = np.poly1d(Z)
    print(f"Relação entre os parâmetros: {coluna2} = {p[0].round(3)} + {p[1].round(3)} {coluna1}")
    
    # Predição de valores
    Y_pred = p(X)
    delta_x = (X.max() - X.min()) / 60
    delta_y = (Y.max() - Y.min()) / 25
    
    # DataFrame auxiliar
    df_aviao = df[['Parameter', coluna1, coluna2]].copy()
    df_aviao['Y_pred'] = Y_pred
    df_aviao.set_index('Parameter', inplace= True)
    
    plt.figure(figsize=(12, 8))
    
    # Plot dos dados
    plt.scatter(X, Y, label='Dados Históricos', color = 'black')

    # Plot da Regressão
    sns.set_theme(style="white")
    sns.lineplot(x=np.sort(X), y=np.sort(Y_pred), color='gray', linestyle='dashed', label=f'Regressão Linear: f(x) = {p[0].round(3)} + {p[1].round(6)}x')
    
    # Legenda nos pontos
    for index, aviao in df_aviao.iterrows():
        x = aviao[coluna1]
        y = aviao[coluna2]
        y_pred = aviao['Y_pred']
        label_x = x - delta_x
        label_y = y + delta_y*0.5 if y > y_pred else y - delta_y
        plt.text(label_x, label_y, str(index), fontsize=10)

    # Legenda e nome dos eixos
    plt.xlabel(coluna1, fontsize=16)
    plt.ylabel(coluna2, fontsize=16)
    plt.legend(fontsize = 14)
    
    ax = plt.gca()

    # Remover os eixos superior e direito
    ax.tick_params(axis='x', top=False)
    ax.tick_params(axis='y', right=False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Setar espessura dos eixos (inferior e esquerdo) e adicionar os traços
    ax.spines['bottom'].set_linewidth(0.6)
    ax.spines['left'].set_linewidth(0.6)
    ax.tick_params(axis='x', bottom=True, direction='in', length=4, width=0.6)
    ax.tick_params(axis='y', left=True, direction='in', length=4, width=0.6)
    
    plt.grid(False)
    
    plt.show()


In [ ]:
# Peso Vazio x MTOW
plotar("MTOW (kg)", "Empty_Weight (kg)")

Relação entre os parâmetros: Empty_Weight (kg) = 32467.473 + -0.121 MTOW (kg)


In [ ]:
# Número de passageiros x MTOW
plotar("Number_of_passengers", "MTOW (kg)")

Relação entre os parâmetros: MTOW (kg) = -211.262 + 460.109 Number_of_passengers


In [ ]:
# Número de passageiros x Comprimento de fuselagem
plotar("Number_of_passengers", "Fuselage_length (m)")

Relação entre os parâmetros: Fuselage_length (m) = 35.278 + -0.043 Number_of_passengers


In [ ]:
# Mach de cruzeiro x Enflechamento da asa
plotar("Cruise_Mach", "Wing_sweep(deg)")

Relação entre os parâmetros: Wing_sweep(deg) = -15.321 + 50.156 Cruise_Mach


In [ ]:
# MTOW x Área da asa
plotar("MTOW (kg)", "Wing_area (ft²)")

Relação entre os parâmetros: Wing_area (ft²) = 16.934 + 0.02 MTOW (kg)


In [ ]:
# MTOW x Range
plotar("MTOW (kg)", "Range (NM)")

Relação entre os parâmetros: Range (NM) = 139.243 + 0.035 MTOW (kg)


In [ ]:
# MTOW x TSFC
plotar("MTOW (kg)", "Total_maximum_thrust (lbf)")


Relação entre os parâmetros: Total_maximum_thrust (lbf) = -3849.936 + 0.731 MTOW (kg)


In [ ]:
# Takeoff_distance x Landing_distance
plotar("Takeoff_distance(m)", "Landing_distance(m)")

Relação entre os parâmetros: Landing_distance(m) = 253.732 + 0.711 Takeoff_distance(m)


In [ ]:
# HT_area x VT_area
plotar("HT_area (m2)", "VT_area (ft2)")

Relação entre os parâmetros: VT_area (ft2) = -6.943 + 8.425 HT_area (m2)
